In [30]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
import math

warnings.filterwarnings('ignore')

plt.rc('font', family='SimHei', size=13)

### 数据格式转换

In [4]:
train_path = '../data/hy_round1_train_20200102'
test_path = '../data/hy_round1_testA_20200102'

In [5]:
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)
print(len(train_files), len(test_files))

7000 2000


In [17]:
# dataframe 转化为 h5文件
def dataframe2hdf(files, path, is_test=False):
    ret = []
    for file in tqdm(files):
        df = pd.read_csv(f'{path}/{file}')
        ret.append(df)
    df = pd.concat(ret)
    df.columns = ['ship', 'x', 'y', 'v', 'd', 'time'] if is_test else ['ship', 'x', 'y', 'v', 'd', 'time', 'type']
    
    
    hdf_name = 'test' if is_test else 'train'
    df.to_hdf(f'../features/{hdf_name}-{int(time.time())}.h5', 'df', mode='w')

In [12]:
dataframe2hdf(train_files, train_path)

100%|█████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:32<00:00, 215.56it/s]


In [18]:
dataframe2hdf(test_files, test_path, True)

100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:08<00:00, 230.40it/s]


### 探索性数据分析（EDA）

In [19]:
import scipy.stats as stats
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

In [20]:
test = pd.read_hdf('../features/test-1578468609.h5')
train = pd.read_hdf('../features/train-1578468401.h5')

In [21]:
test.sample(5)

,ship,x,y,v,d,time
388,8891,6.331277e+06,5.392778e+06,0.11,0,1107 16:20:55
129,8626,6.655768e+06,5.746294e+06,1.30,87,1105 23:01:30
164,7649,6.109209e+06,5.114764e+06,0.05,0,1109 05:50:58
9,8274,6.326124e+06,5.163422e+06,1.03,199,1030 22:11:43
114,7614,6.182676e+06,5.193037e+06,0.22,199,1120 04:38:43


In [22]:
train.sample(5)

,ship,x,y,v,d,time,type
113,6742,6.265082e+06,5.251775e+06,0.32,143,1102 15:00:23,拖网
35,2448,5.781048e+06,4.903953e+06,1.67,261,1113 17:59:24,拖网
315,5911,6.109107e+06,5.114766e+06,0.05,0,1104 06:44:41,拖网
346,1939,6.593804e+06,5.602399e+06,2.70,25,1111 10:26:12,刺网
349,6068,6.246528e+06,5.241480e+06,0.11,163,1104 13:12:57,拖网


In [24]:
type_dict = {'围网':0, '拖网': 1, '刺网': 2}
train['type'] = train['type'].apply(lambda x : type_dict[x])

In [25]:
train.sample(5)

,ship,x,y,v,d,time,type
163,3066,6.375734e+06,5.263153e+06,0.38,206,1112 18:17:47,1
367,770,6.180239e+06,5.192645e+06,0.11,0,1031 20:11:55,2
145,1819,6.160334e+06,5.142043e+06,3.18,20,1112 19:55:40,1
162,6558,6.117145e+06,5.131466e+06,0.00,107,1029 19:26:00,0
69,2477,6.118555e+06,5.130667e+06,0.11,0,1102 23:51:38,1


In [76]:
# 解析时间
def extract_dt(df):
    df['time'] = pd.to_datetime(df['time'], format='%m%d %H:%M:%S')
    df['month'] = df['time'].dt.month
    df['day'] = df['time'].dt.day
    df['date'] = df['time'].dt.date
    df['hour'] = df['time'].dt.hour
    df['weekday'] = df['time'].dt.weekday
    return df

def group_feature(df, key, target, aggs):
    agg_dict = {}
    for ag in aggs:
        agg_dict[f'{target}_{ag}'] = ag
    print(agg_dict)
    t = df.groupby(key)[target].agg(agg_dict).reset_index()
    return t

def extract_feature(df, train):
    # df: 原始数据 train: 特征数据
    # df: data, key: feature row, target: , aggs: twice feature.
    # skew: 偏度，是统计数据分布偏斜方向的程度的度量，是统计数据分布非对称程度的数字特征。
    # $skew(X)=E(\frac{X-\mu}{\sigma}^3)$
    t = group_feature(df, 'ship', 'x', ['max', 'min', 'mean', 'std', 'skew', 'sum', 'count'])
    train = pd.merge(train, t, on='ship', how='left')
    t = group_feature(df, 'ship','y',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    t = group_feature(df, 'ship','v',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    t = group_feature(df, 'ship','d',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    
    # 区域确定
    train['x_max_x_min'] = train['x_max'] - train['x_min']
    train['y_max_y_min'] = train['y_max'] - train['y_min']
    train['y_max_x_min'] = train['y_max'] - train['x_min']
    train['x_max_y_min'] = train['x_max'] - train['y_min']
    
    # 坡度和面积
    train['slope'] = train['y_max_y_min'] / np.where(train['x_max_x_min']==0, 0.001, train['x_max_x_min'])
    train['area'] = train['x_max_x_min'] * train['y_max_y_min']
    
    # 路程
    df = df.sort_values('time')
    df_diff = df.diff(1).iloc[1:]
    df_diff['time_seconds'] = df_diff['time'].dt.total_seconds()
    df_diff['dis'] = np.sqrt(df_diff['x']**2 + df_diff['y']**2)
    t = group_feature(df_diff, 'ship','dis',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    
    # 计算斜率
    df_diff['aslope'] = df_diff['y'] / df_diff['x'].apply(lambda x: 1.0 if math.isclose(0.0, x) else x)
    t = group_feature(df_diff, 'ship','aslope',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    
    # 计算速度
    df_diff['aspeed'] = df_diff['dis'] / df_diff['time_seconds'].apply(lambda x: 1.0 if math.isclose(0.0, x) else x)
    t = group_feature(df_diff, 'ship','aspeed',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    
    # value_counts()函数可以对Series里面的每个值进行计数并且排序
    # 统计每条船每天，在哪个小时出海的频率最大
    mode_hour = df.groupby('ship')['hour'].agg(lambda x:x.value_counts().index[0]).to_dict()
    train['mode_hour'] = train['ship'].map(mode_hour)
    
    # 小时
    t = group_feature(df, 'ship','hour',['max','min'])
    train = pd.merge(train, t, on='ship', how='left')
    
    # 某船工作的时间/天数
    hour_nunique = df.groupby('ship')['hour'].nunique().to_dict()
    date_nunique = df.groupby('ship')['date'].nunique().to_dict()
    train['hour_nunique'] = train['ship'].map(hour_nunique)
    train['date_nunique'] = train['ship'].map(date_nunique)
    
    # 统计每条船总共工作时长
    t = df.groupby('ship')['time'].agg({'diff_time':lambda x:np.max(x)-np.min(x)}).reset_index()
    t['diff_day'] = t['diff_time'].dt.days
    t['diff_second'] = t['diff_time'].dt.seconds
    train = pd.merge(train, t, on='ship', how='left')
    return train

In [77]:
## 解析时间
train = extract_dt(train)
test = extract_dt(test)

In [78]:
train_label = train.drop_duplicates('ship')
test_label = test.drop_duplicates('ship')

In [79]:
train_label = extract_feature(train, train_label)

{'x_max': 'max', 'x_min': 'min', 'x_mean': 'mean', 'x_std': 'std', 'x_skew': 'skew', 'x_sum': 'sum', 'x_count': 'count'}
{'y_max': 'max', 'y_min': 'min', 'y_mean': 'mean', 'y_std': 'std', 'y_skew': 'skew', 'y_sum': 'sum'}
{'v_max': 'max', 'v_min': 'min', 'v_mean': 'mean', 'v_std': 'std', 'v_skew': 'skew', 'v_sum': 'sum'}
{'d_max': 'max', 'd_min': 'min', 'd_mean': 'mean', 'd_std': 'std', 'd_skew': 'skew', 'd_sum': 'sum'}
{'dis_max': 'max', 'dis_min': 'min', 'dis_mean': 'mean', 'dis_std': 'std', 'dis_skew': 'skew', 'dis_sum': 'sum'}
{'aslope_max': 'max', 'aslope_min': 'min', 'aslope_mean': 'mean', 'aslope_std': 'std', 'aslope_skew': 'skew', 'aslope_sum': 'sum'}
{'aspeed_max': 'max', 'aspeed_min': 'min', 'aspeed_mean': 'mean', 'aspeed_std': 'std', 'aspeed_skew': 'skew', 'aspeed_sum': 'sum'}
{'hour_max': 'max', 'hour_min': 'min'}


In [80]:
test_label = extract_feature(test, test_label)

{'x_max': 'max', 'x_min': 'min', 'x_mean': 'mean', 'x_std': 'std', 'x_skew': 'skew', 'x_sum': 'sum', 'x_count': 'count'}
{'y_max': 'max', 'y_min': 'min', 'y_mean': 'mean', 'y_std': 'std', 'y_skew': 'skew', 'y_sum': 'sum'}
{'v_max': 'max', 'v_min': 'min', 'v_mean': 'mean', 'v_std': 'std', 'v_skew': 'skew', 'v_sum': 'sum'}
{'d_max': 'max', 'd_min': 'min', 'd_mean': 'mean', 'd_std': 'std', 'd_skew': 'skew', 'd_sum': 'sum'}
{'dis_max': 'max', 'dis_min': 'min', 'dis_mean': 'mean', 'dis_std': 'std', 'dis_skew': 'skew', 'dis_sum': 'sum'}
{'aslope_max': 'max', 'aslope_min': 'min', 'aslope_mean': 'mean', 'aslope_std': 'std', 'aslope_skew': 'skew', 'aslope_sum': 'sum'}
{'aspeed_max': 'max', 'aspeed_min': 'min', 'aspeed_mean': 'mean', 'aspeed_std': 'std', 'aspeed_skew': 'skew', 'aspeed_sum': 'sum'}
{'hour_max': 'max', 'hour_min': 'min'}


In [35]:
resumetable(train_label)

Dataset Shape: (7000, 69)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,ship,int64,0,7000,0,1,10,12.77
1,x,float64,0,4555,6.15204e+06,6.07625e+06,6.32103e+06,11.50
2,y,float64,0,4555,5.12487e+06,5.06174e+06,5.24281e+06,11.50
3,v,float64,0,169,2.59,3.99,4.48,5.40
4,d,int64,0,360,102,278,213,6.55
...,...,...,...,...,...,...,...,...
64,hour_nunique,int64,0,19,24,24,24,0.13
65,date_nunique,int64,0,4,4,4,4,1.02
66,diff_time,timedelta64[ns],0,1841,2 days 23:48:51,2 days 23:39:47,2 days 23:33:53,9.80
67,diff_day,int64,0,3,2,2,2,0.08


In [36]:
resumetable(test_label)

Dataset Shape: (2000, 68)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,ship,int64,0,2000,7000,7001,7002,10.97
1,x,float64,0,1549,7.11885e+06,6.24673e+06,6.74145e+06,10.35
2,y,float64,0,1549,5.91828e+06,5.24115e+06,5.56669e+06,10.35
3,v,float64,0,142,0.11,2.7,0.49,5.47
4,d,int64,0,321,0,173,248,6.46
...,...,...,...,...,...,...,...,...
63,hour_nunique,int64,0,6,24,24,24,0.04
64,date_nunique,int64,0,3,4,3,3,1.00
65,diff_time,timedelta64[ns],0,930,2 days 23:37:33,2 days 23:55:15,2 days 23:45:56,9.24
66,diff_day,int64,0,3,2,2,2,0.05


In [81]:
train_label['diff_time'] = train_label['diff_time']/np.timedelta64(1, 's')
test_label['diff_time'] = test_label['diff_time']/np.timedelta64(1, 's')

In [82]:
features = [x for x in train_label.columns if x not in ['ship','type','time','x','date','y','v','d']]
target = 'type'

In [39]:
print(len(features), ','.join(features))

61 month,day,hour,weekday,x_max,x_min,x_mean,x_std,x_skew,x_sum,x_count,y_max,y_min,y_mean,y_std,y_skew,y_sum,v_max,v_min,v_mean,v_std,v_skew,v_sum,d_max,d_min,d_mean,d_std,d_skew,d_sum,x_max_x_min,y_max_y_min,y_max_x_min,x_max_y_min,slope,area,dis_max,dis_min,dis_mean,dis_std,dis_skew,dis_sum,aslope_max,aslope_min,aslope_mean,aslope_std,aslope_skew,aslope_sum,aspeed_max,aspeed_min,aspeed_mean,aspeed_std,aspeed_skew,aspeed_sum,mode_hour,hour_max,hour_min,hour_nunique,date_nunique,diff_time,diff_day,diff_second


In [83]:
train_label.dtypes.to_dict()

{'ship': dtype('int64'),
 'x': dtype('float64'),
 'y': dtype('float64'),
 'v': dtype('float64'),
 'd': dtype('int64'),
 'time': dtype('<M8[ns]'),
 'type': dtype('int64'),
 'month': dtype('int64'),
 'day': dtype('int64'),
 'date': dtype('O'),
 'hour': dtype('int64'),
 'weekday': dtype('int64'),
 'x_max': dtype('float64'),
 'x_min': dtype('float64'),
 'x_mean': dtype('float64'),
 'x_std': dtype('float64'),
 'x_skew': dtype('float64'),
 'x_sum': dtype('float64'),
 'x_count': dtype('int64'),
 'y_max': dtype('float64'),
 'y_min': dtype('float64'),
 'y_mean': dtype('float64'),
 'y_std': dtype('float64'),
 'y_skew': dtype('float64'),
 'y_sum': dtype('float64'),
 'v_max': dtype('float64'),
 'v_min': dtype('float64'),
 'v_mean': dtype('float64'),
 'v_std': dtype('float64'),
 'v_skew': dtype('float64'),
 'v_sum': dtype('float64'),
 'd_max': dtype('int64'),
 'd_min': dtype('int64'),
 'd_mean': dtype('float64'),
 'd_std': dtype('float64'),
 'd_skew': dtype('float64'),
 'd_sum': dtype('int64'),
 'x

In [84]:
train_label.sample().to_dict()

{'ship': {727: 1652},
 'x': {727: 6182677.281026656},
 'y': {727: 5193146.03309477},
 'v': {727: 0.0},
 'd': {727: 327},
 'time': {727: Timestamp('1900-11-06 23:52:46')},
 'type': {727: 1},
 'month': {727: 11},
 'day': {727: 6},
 'date': {727: datetime.date(1900, 11, 6)},
 'hour': {727: 23},
 'weekday': {727: 1},
 'x_max': {727: 6209150.66115042},
 'x_min': {727: 6182376.460210676},
 'x_mean': {727: 6183272.120403779},
 'x_std': {727: 3484.696263838164},
 'x_skew': {727: 6.005879550062226},
 'x_sum': {727: 2003380167.0108244},
 'x_count': {727: 324},
 'y_max': {727: 5193688.561652083},
 'y_min': {727: 5151962.271482084},
 'y_mean': {727: 5192274.549629206},
 'y_std': {727: 5263.19988533277},
 'y_skew': {727: -6.142825343459828},
 'y_sum': {727: 1682296954.0798628},
 'v_max': {727: 10.09},
 'v_min': {727: 0.0},
 'v_mean': {727: 0.4570370370370366},
 'v_std': {727: 1.2948434950381404},
 'v_skew': {727: 4.918274059116307},
 'v_sum': {727: 148.07999999999984},
 'd_max': {727: 352},
 'd_min

### lightgb模型

- 

In [96]:
params = {
    'n_estimators': 5000,
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'early_stopping_rounds': 100,
    'max_depth': 6,     ###   根据问题来定咯，由于我的数据集不是很大，所以选择了一个适中的值，其实4-10都无所谓。
    'num_leaves':50,  ###   由于lightGBM是leaves_wise生长，官方说法是要小于2^max_depth
    'learning_rate': 0.1,
#     'subsample':0.8 ,          ###  数据采样
#     'colsample_bytree': 0.8  ###  特征采样
}

In [97]:
train_label[features].to_hdf(f'../features/train-final-{int(time.time())}.h5', 'df', mode='w')

In [98]:
test_label[features].to_hdf(f'../features/test-final-{int(time.time())}.h5', 'df', mode='w')

In [88]:
from sklearn.metrics import confusion_matrix
import itertools
# 绘制混淆矩阵
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Input
    - cm : 计算出的混淆矩阵的值
    - classes : 混淆矩阵中每一行每一列对应的列
    - normalize : True:显示百分比, False:显示个数
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [99]:
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X = train_label[features].copy()
y = train_label[target]
models = []
pred = np.zeros((len(test_label),3))
oof = np.zeros((len(X), 3))
conf_mat = []
for index, (train_idx, val_idx) in enumerate(fold.split(X, y)):

    train_set = lgb.Dataset(X.iloc[train_idx], y.iloc[train_idx])
    val_set = lgb.Dataset(X.iloc[val_idx], y.iloc[val_idx])

    model = lgb.train(params, train_set, valid_sets=[train_set, val_set], verbose_eval=100)
    models.append(model)
    val_pred = model.predict(X.iloc[val_idx])
    oof[val_idx] = val_pred
    val_y = y.iloc[val_idx]
    val_pred = np.argmax(val_pred, axis=1)
    print(index, 'val f1', metrics.f1_score(val_y, val_pred, average='macro'))
    # 0.8695539641133697
    # 0.8866211724839532
#     print(val_y, val_pred)
    conf_mat.append(confusion_matrix(val_y, val_pred))
#     plot_confusion_matrix(conf_mat, classes=classes)
    test_pred = model.predict(test_label[features])
    pred += test_pred/5

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.119513	valid_1's multi_logloss: 0.313709
[200]	training's multi_logloss: 0.0386271	valid_1's multi_logloss: 0.29941
Early stopping, best iteration is:
[187]	training's multi_logloss: 0.0446967	valid_1's multi_logloss: 0.297924
0 val f1 0.8418069794258732
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.116255	valid_1's multi_logloss: 0.310344
[200]	training's multi_logloss: 0.0404425	valid_1's multi_logloss: 0.306199
Early stopping, best iteration is:
[138]	training's multi_logloss: 0.0773307	valid_1's multi_logloss: 0.303304
1 val f1 0.8326390068022888
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.114982	valid_1's multi_logloss: 0.319753
[200]	training's multi_logloss: 0.0369149	valid_1's multi_logloss: 0.311025
Early stopping, best iteration is:
[173]	training's multi_logloss: 0.0493796	valid_1

In [100]:
oof_t = np.argmax(oof, axis=1)
print('oof f1', metrics.f1_score(oof_t, y, average='macro'))
score = metrics.f1_score(oof_t, y, average='macro')

oof f1 0.8401984898296234


In [101]:
type_map_rev = {0: '拖网', 1: '围网', 2: '刺网'}

pred_max = np.argmax(pred, axis=1)
sub = test_label[['ship']]
sub['pred'] = pred_max

print(sub['pred'].value_counts(1))
sub['pred'] = sub['pred'].map(type_map_rev)
sub.to_csv(f'../results/result-{score}.csv', index=None, header=None)

1    0.6905
0    0.2030
2    0.1065
Name: pred, dtype: float64


In [102]:
ret = []
for index, model in enumerate(models):
    df = pd.DataFrame()
    df['name'] = model.feature_name()
    df['score'] = model.feature_importance()
    df['fold'] = index
    ret.append(df)
    
df = pd.concat(ret)

In [103]:
df = df.groupby('name', as_index=False)['score'].mean()
df = df.sort_values(['score'], ascending=False)

In [104]:
df.to_dict()

{'name': {54: 'y_max_x_min',
  53: 'y_max',
  49: 'x_min',
  47: 'x_max_y_min',
  41: 'v_std',
  57: 'y_min',
  45: 'x_max',
  40: 'v_skew',
  50: 'x_skew',
  56: 'y_mean',
  48: 'x_mean',
  58: 'y_skew',
  36: 'slope',
  17: 'd_std',
  38: 'v_mean',
  14: 'd_mean',
  37: 'v_max',
  23: 'diff_time',
  42: 'v_sum',
  4: 'aslope_skew',
  51: 'x_std',
  16: 'd_skew',
  27: 'dis_skew',
  18: 'd_sum',
  3: 'aslope_min',
  46: 'x_max_x_min',
  59: 'y_std',
  55: 'y_max_y_min',
  0: 'area',
  28: 'dis_std',
  26: 'dis_min',
  5: 'aslope_std',
  2: 'aslope_mean',
  10: 'aspeed_skew',
  44: 'x_count',
  52: 'x_sum',
  9: 'aspeed_min',
  25: 'dis_mean',
  11: 'aspeed_std',
  8: 'aspeed_mean',
  24: 'dis_max',
  6: 'aslope_sum',
  1: 'aslope_max',
  60: 'y_sum',
  34: 'mode_hour',
  7: 'aspeed_max',
  13: 'd_max',
  29: 'dis_sum',
  20: 'day',
  12: 'aspeed_sum',
  39: 'v_min',
  22: 'diff_second',
  43: 'weekday',
  30: 'hour',
  19: 'date_nunique',
  35: 'month',
  15: 'd_min',
  21: 'diff_day'

In [105]:
train_label.shape

(7000, 69)